<a href="https://colab.research.google.com/github/jjjonathan14/crowdhuman_tensorRT/blob/main/jesus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install einops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
import torch.nn.functional as F
from torch import nn
from torch import Tensor
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce

class PatchEmbedding(nn.Module):
    def __init__(self, in_channels: int = 3, patch_size: int = 16, emb_size: int = 768, img_size: int = 224):
        self.patch_size = patch_size
        super().__init__()
        self.projection = nn.Sequential(
            # using a conv layer instead of a linear one -> performance gains
            nn.Conv2d(in_channels, emb_size, kernel_size=patch_size, stride=patch_size),
            Rearrange('b e (h) (w) -> b (h w) e'),
        )
        self.cls_token = nn.Parameter(torch.randn(1 ,1, emb_size))
        self.positions = nn.Parameter(torch.randn((img_size // patch_size) **2 + 1, emb_size))

    def forward(self, x: Tensor) -> Tensor:
        b, _, _, _ = x.shape

        x = self.projection(x)
        cls_tokens = repeat(self.cls_token, '() n e -> b n e', b=b)
        # prepend the cls token to the input
        x = torch.cat([cls_tokens, x], dim=1)
        # add position embedding
        x += self.positions
        return x

class MultiHeadAttention(nn.Module):
    def __init__(self, emb_size: int = 768, num_heads: int = 8, dropout: float = 0):
        super().__init__()
        self.emb_size = emb_size
        self.num_heads = num_heads
        # fuse the queries, keys and values in one matrix
        self.qkv = nn.Linear(emb_size, emb_size * 3)
        self.att_drop = nn.Dropout(dropout)
        self.projection = nn.Linear(emb_size, emb_size)

    def forward(self, x : Tensor, mask: Tensor = None) -> Tensor:
        # split keys, queries and values in num_heads
        qkv = rearrange(self.qkv(x), "b n (h d qkv) -> (qkv) b h n d", h=self.num_heads, qkv=3)
        queries, keys, values = qkv[0], qkv[1], qkv[2]
        # sum up over the last axis
        energy = torch.einsum('bhqd, bhkd -> bhqk', queries, keys) # batch, num_heads, query_len, key_len
        if mask is not None:
            fill_value = torch.finfo(torch.float32).min
            energy.mask_fill(~mask, fill_value)

        scaling = self.emb_size ** ( 1 /2)
        att = F.softmax(energy, dim=-1) / scaling
        att = self.att_drop(att)
        # sum up over the third axis
        out = torch.einsum('bhal, bhlv -> bhav ', att, values)
        out = rearrange(out, "b h n d -> b n (h d)")
        out = self.projection(out)
        return out

class ResidualAdd(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn

    def forward(self, x, **kwargs):
        res = x
        x = self.fn(x, **kwargs)
        x += res
        return x

class FeedForwardBlock(nn.Sequential):
    def __init__(self, emb_size: int =768, expansion: int = 4, drop_p: float = 0.):
        super().__init__(
            nn.Linear(emb_size, expansion * emb_size),
            nn.GELU(),
            nn.Dropout(drop_p),
            nn.Linear(expansion * emb_size, emb_size),
        )

class TransformerEncoderBlock(nn.Sequential):
    def __init__(self,
                 emb_size: int = 768,
                 drop_p: float = 0.,
                 forward_expansion: int = 4,
                 forward_drop_p: float = 0.,
                 ** kwargs):
        super().__init__(
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                MultiHeadAttention(emb_size, **kwargs),
                nn.Dropout(drop_p)
            )),
            ResidualAdd(nn.Sequential(
                nn.LayerNorm(emb_size),
                FeedForwardBlock(
                    emb_size, expansion=forward_expansion, drop_p=forward_drop_p),
                nn.Dropout(drop_p)
            )
            ))

class TransformerEncoder(nn.Sequential):
    def __init__(self, depth: int = 12, **kwargs):
        super().__init__(*[TransformerEncoderBlock(**kwargs) for _ in range(depth)])

class ClassificationHead(nn.Sequential):
    def __init__(self, emb_size: int = 768, n_classes: int = 1000):
        super().__init__(
            Reduce('b n e -> b e', reduction='mean'),
            nn.LayerNorm(emb_size),
            nn.Linear(emb_size, n_classes))

class ViT(nn.Sequential):
    def __init__(self,
                 in_channels: int = 64,
                 patch_size: int = 16,
                 emb_size: int = 768,
                 img_size: int = 56,
                 depth: int = 12,
                 n_classes: int = 1000,
                 **kwargs):
        super().__init__(
            PatchEmbedding(in_channels, patch_size, emb_size, img_size),
            TransformerEncoder(depth, emb_size=emb_size, **kwargs),
            ClassificationHead(emb_size, n_classes)
        )

In [4]:
# -*- coding: utf-8 -*-
"""j-Hybrid.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Ola5OebLU1VOKCpQ2RvDJVRj8Myhpugk
"""

# !pip install einops
import random

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
import numpy as np
import torch
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch
import math
import numpy as np
import matplotlib.pyplot as plt


def batch_plot(img,n, h, w):
  fig = plt.figure(figsize=(n, n))
  columns = n
  rows = n
  for i in range(1, columns*rows):

      fig.add_subplot(rows, columns, i)
      plt.imshow(img[i])
  plt.show()



arr = torch.rand(1, 1, 28, 28)

class Hybrid(nn.Module):
    """
    block: A sub module
    """
    def __init__(self, inchannels = 1,n_classes = 10, embedding = 96):
        super(Hybrid, self).__init__()
        # self.conv1 = nn.Conv2d(inchannels, 64, kernel_size = 7, stride = 2, padding = 3,
        #                        bias = False)
        # self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # self.patchEmbedding1 = PatchEmbedding(inchannels, 8, embedding, 224)
        self.transformerEncoder = TransformerEncoderBlock(embedding, 0.0, 4, 0.0)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # self.conv2 = nn.Conv2d(64, 128, kernel_size=1, bias=False)
        # self.bn2 = nn.BatchNorm2d(128)
        # self.patchEmbedding2 = PatchEmbedding(64, 8, embedding, 56)
        # self.conv22 = nn.Conv2d(128, 128, kernel_size=1, bias=False)
        # self.conv222 = nn.Conv2d(128, 128, kernel_size=3, padding=1, bias=False)

        self.conv3 = nn.Conv2d(inchannels, 16, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(16)
        self.patchEmbedding3 = PatchEmbedding(inchannels, 8, embedding, 28)
        self.conv33 = nn.Conv2d(16, 16, kernel_size=1, bias=False)
        self.conv333 = nn.Conv2d(16, 16, kernel_size=3, padding=1, bias=False)
        



        self.conv4 = nn.Conv2d(16, 32, kernel_size=1, bias=False)
        self.bn4 = nn.BatchNorm2d(32)
        self.patchEmbedding4 = PatchEmbedding(16, 8, embedding, 14)
        self.conv44 = nn.Conv2d(32, 32, kernel_size=1, bias=False)
        self.conv444 = nn.Conv2d(32, 32, kernel_size=3, padding=1, bias=False)


        self.conv5 = nn.Conv2d(32, 64, kernel_size=1, bias=False)
        self.bn5 = nn.BatchNorm2d(64)
        self.patchEmbedding5 = PatchEmbedding(32, 3, embedding, 7)
        self.conv55 = nn.Conv2d(64, 64, kernel_size=1, bias=False)
        self.conv555 = nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False)

        self.nn1 = nn.Linear(1024, 512)
        self.nn2 = nn.Linear(512, n_classes)


        self.nny1_1 = nn.Linear(75360, 7500)
        self.nny1_2 = nn.Linear(7500, 480)
        self.nny2 = nn.Linear(4800, 480)
        self.nny3 = nn.Linear(960, 480)
        self.nny4 = nn.Linear(192, 480)

        self.nnR = nn.Linear(480, 480)
        self.nnY1 = nn.Linear(480, 48)
        self.nnY2 = nn.Linear(48, n_classes)
        self.sigmoid = nn.Sigmoid()
        #self.classificationhead = ClassificationHead(emb_size = 768, n_classes = 1000)

        self.dropout = nn.Dropout(p=0.1)


    def forward(self, x):
        # y = x
        # x = self.conv1(x)
        # x = self.bn1(x)
        # x = self.relu(x)
        # x = self.maxpool(x)
        # y = self.patchEmbedding1(y)
        # y1 = self.transformerEncoder(y)
        # y1 = self.transformerEncoder(y1)
        # y1 = torch.flatten(y1, start_dim=1)
        # y1 = self.nny1_1(y1)
        # y1 = self.nny1_2(y1)


        # y = x
        # x = self.conv2(x)
        # x = self.bn2(x)
        # z1 = x
        # r = x
        # z1 = self.conv22(z1)
        # z1 = self.bn2(z1)
        # z1 = self.conv222(z1)
        # z1 = self.bn2(z1)
        # z1 = self.conv22(z1)
        # z1 = self.bn2(z1)
        # x = r + z1
        # x = self.maxpool1(x)
        # y = self.patchEmbedding2(y)
        # y2 = self.transformerEncoder(y)
        # y2 = self.transformerEncoder(y2)
        # y2 = torch.flatten(y2, start_dim=1)
        # y2 = self.nny2(y2)
        # y2 = y1 + y1
        # y2 = self.nnR(y2)


        y = x
        #print('x input', x.shape)
        x = self.conv3(x)
        x = self.bn3(x)
        t = x
        #print('conv3 shape', x.shape, x[0].shape)
        #batch_plot(t[0].detach().cpu().numpy(), 16, 28, 28)
        z2 = x
        r = x
        z2 = self.conv33(z2)
        z2 = self.relu(self.bn3(z2))
        z2 = self.conv333(z2)
        z2 = self.relu(self.bn3(z2))
        z2 = self.conv33(z2)
        z2 = self.relu(self.bn3(z2))
        x = r + z2
        x = self.maxpool1(x)
        y = self.patchEmbedding3(y)
        y3 = self.transformerEncoder(y)
        y3 = self.transformerEncoder(y3)
        y3 = torch.flatten(y3, start_dim=1)
        #print('y3 transformer', y3.shape)
        y3 = self.dropout(self.relu(self.nny3(y3)))
        #y3 = y2 + y3
        y3 = self.dropout(self.relu(self.nnR(y3)))


        y = x
        x = self.conv4(x)
        x = self.bn4(x)
        z3 = x
        r = x
        z3 = self.conv44(z3)
        z3 = self.relu(self.bn4(z3))
        z3 = self.conv444(z3)
        z3 = self.relu(self.bn4(z3))
        z3 = self.conv44(z3)
        z3 = self.relu(self.bn4(z3))
        x = r + z3
        x = self.maxpool1(x)
        y = self.patchEmbedding4(y)
        y4 = self.transformerEncoder(y)
        y4 = self.transformerEncoder(y4)
        y4 = torch.flatten(y4, start_dim=1)
        #print('y4 transformer', y4.shape)
        y4 = self.dropout(self.relu(self.nny4(y4)))
        y4 = y3 + y4
        y4 = self.dropout(self.relu(self.nnR(y4)))


        y = x
        x = self.conv5(x)
        x = self.bn5(x)
        z4 = x
        r = x
        z4 = self.conv55(z4)
        z4 = self.relu(self.bn5(z4))
        t1 = x
        #batch_plot(t1[0].detach().cpu().numpy(), 32, 7, 7)
        z4 = self.conv555(z4)
        z4 = self.relu(self.bn5(z4))
        z4 = self.conv55(z4)
        z4 = self.relu(self.bn5(z4))
        x = r + z4
        x = self.maxpool1(x)
        X = torch.flatten(x, start_dim=1)
        #print('X', X.shape)
        y = self.patchEmbedding5(y)
        y5 = self.transformerEncoder(y)
        y5 = self.transformerEncoder(y5)
        y5 = torch.flatten(y5, start_dim=1)
        #print('y5 transformer encoder', y5.shape)

        y5 = y4 + y5
        y5 = self.dropout(self.relu(self.nnR(y5)))

        Y = self.dropout(self.relu(self.nnY1(y5)))
        Y = self.dropout(self.relu(self.nnY2(Y)))
        Y = self.sigmoid(Y)


        X = self.dropout(self.relu(self.nn1(X)))
        X = self.sigmoid(self.nn2(X))


        Z = X + Y

        return X, Y, Z



In [5]:
model = Hybrid()

In [6]:
res = model(arr)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
# from jesus import Hybrid
import torch.optim as optim
import cv2
from tqdm import tqdm
import time

torch.cuda.empty_cache()

batch_size = 1
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               #torchvision.transforms.Resize((224,224)),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,)),
                             ])),
  batch_size=batch_size, shuffle=False)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               #torchvision.transforms.Resize((224,224)),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size, shuffle=False)
classes = ('0', '1', '2', '3',
           '4', '5', '6', '7', '8', '9')


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

model = Hybrid()
model.cuda()

criterion = nn.CrossEntropyLoss()
criterion_hybrid = nn.HuberLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(100):  # loop over the dataset multiple times
    
    t = epoch % 9
    if epoch < 10:
      n = 10
      
    elif epoch < 20:
      n =7
      
    elif epoch < 30:
      n = 5
      
    else:
      n = 3
      
    for memorize in range(n):
          if epoch < 10:
            optimizer = optim.Adam(model.parameters(), lr=0.01*memorize)

          elif epoch < 20:
            optimizer = optim.Adam(model.parameters(), lr=0.001*memorize)

          elif epoch < 30:
            optimizer = optim.Adam(model.parameters(), lr=0.0001*memorize)

          else:
            optimizer = optim.Adam(model.parameters(), lr=0.0001)

          running_loss_hybrid = 0.0
          running_loss_cnn = 0.0
          running_loss_vit = 0.0
          running_loss_ssl = 0.0
          count = 0
          print('memorize', memorize)
          for i, data in tqdm(enumerate(train_loader, 0)):
              # get the inputs; data is a list of [inputs, labels]
              if i % 10000 == 9999:    # print every 2000 mini-batches
                  print(f'[{epoch + 1}, {i + 1:5d}] count {count} loss hybrid: {running_loss_hybrid} loss ssl: {running_loss_ssl}')
                  with open('/content/drive/MyDrive/self_learning/result.txt','a') as f:
                    f.write(f'[{epoch + 1}, {i + 1:5d}] count {count} loss hybrid:  {running_loss_hybrid} loss ssl:  {running_loss_ssl}')
                  running_loss_hybrid = 0.0
                  running_loss_cnn = 0.0
                  running_loss_vit = 0.0
                  running_loss_ssl = 0.0

                  count = 0

                  # correct = 0
                  # total = 0
                  # # since we're not training, we don't need to calculate the gradients for our outputs
                  # with torch.no_grad():
                  #     for data in test_loader:
                  #         images, labels = data
                  #         inputs, labels = inputs.cuda(), labels.cuda()
                  #         # calculate outputs by running images through the network
                  #         outputs = Hybrid(images)
                  # #         # the class with the highest energy is what we choose as prediction
                  #         _, predicted = torch.max(outputs.data, 1)
                  #         total += labels.size(0)
                  #         correct += (predicted == labels).sum().item()

                  # print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')
                  # with open('/content/drive/MyDrive/self_learning/accuracy','a') as w:
                  #   w.write(f'[{epoch + 1}, {i + 1:5d}] count {count} loss hybrid: {running_loss_hybrid / count:.3f} loss cnn: {running_loss_cnn / count:.3f} loss vit: {running_loss_vit / count:.3f} \n ')
                  

              inputs, labels = data
              if not labels == t:
                continue
              count += 1
              inputs, labels = inputs.cuda(), labels.cuda()

              # zero the parameter gradients
              # optimizer.zero_grad()
              # forward + backward + optimize
              #optimizer.zero_grad()
              X, Y, Z = model(inputs)
              labels = labels.to(torch.float32)
              loss1 = criterion_hybrid(Z, labels)
              loss1.backward()
              optimizer.step()
              running_loss_hybrid += loss1.item()

              # forward + backward + optimize
              #optimizer.zero_grad()
              # X, Y, Z = model(inputs)
              # loss2 = criterion(X, Z)
              # loss2.backward()
              # optimizer.step()
              # running_loss_cnn += loss2.item()

              # # forward + backward + optimize
              # #optimizer.zero_grad()
              # X, Y, Z = model(inputs)
              # loss3 = criterion(Y, X)
              # loss3.backward()
              # optimizer.step()
              # running_loss_vit += loss3.item()
#self supervisied 
              # X, Y, Z = model(inputs)
              # loss4 = criterion(Y, X)
              # #print('loss ssl', loss4)
              # loss4.backward()
              # optimizer.step()
              # running_loss_ssl += loss4.item()

              # print statistics
              


memorize 0


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:982: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.huber_loss(input, target, reduction=self.reduction, delta=self.delta)
9984it [00:36, 299.12it/s]

[1, 10000] count 1001 loss hybrid: 514.4837219119072 loss ssl: 0.0


20096it [01:05, 393.91it/s]

[1, 20000] count 993 loss hybrid: 509.7196875810623 loss ssl: 0.0


...

40039it [01:47, 362.99it/s]

[1, 40000] count 963 loss hybrid: 120.375 loss ssl: 0.0


50067it [02:15, 339.88it/s]

[1, 50000] count 1008 loss hybrid: 126.0 loss ssl: 0.0


51901it [02:20, 336.36it/s]